In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


####Importing Libraries

In [0]:
import pandas as pd
import numpy as np 
from sklearn import preprocessing

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Customer-Retention-data/train_final.csv')
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Customer-Retention-data/test_final.csv')

In [0]:
train_features = train.drop(['new.returningCustomer'],axis=1)
train_labels = train['new.returningCustomer']
test_features = test.drop(['new.returningCustomer'],axis=1)
test_labels = test['new.returningCustomer']


In [0]:
train_features.head(5)

,channelGrouping,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime,year,month,day,country_dem,latitude,longitude
0,Referral,Chrome,desktop,False,Chrome OS,8.934117e+18,Cupertino,Americas,United States,unknown.unknown,California,Northern America,-1.0,2,0.0,2.0,2.0,28.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),False,UNK,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com,1508176307,6,1508176307,-0.068615,0.897805,-0.866025,645204,37.090240,-95.712891
1,Direct,Chrome,mobile,True,Android,7.992466e+18,not available in demo dataset,Americas,United States,windjammercable.net,not available in demo dataset,Northern America,-1.0,2,1.0,2.0,1.0,38.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),True,UNK,(none),UNK,(direct),1508201613,1,1508201613,-0.068615,0.897805,-0.866025,645204,37.090240,-95.712891
2,Organic Search,Chrome,desktop,False,Windows,9.075656e+18,not available in demo dataset,Asia,Turkey,unknown.unknown,not available in demo dataset,Western Asia,-1.0,2,1.0,2.0,1.0,1.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),False,(not provided),organic,UNK,google,1508169851,1,1508169851,-0.068615,0.897805,-0.866025,26282,38.963745,35.243322
3,Organic Search,Chrome,desktop,False,Windows,6.960673e+18,not available in demo dataset,Americas,Mexico,prod-infinitum.com.mx,not available in demo dataset,Central America,-1.0,2,1.0,2.0,1.0,52.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),False,(not provided),organic,UNK,google,1508190552,1,1508190552,-0.068615,0.897805,-0.866025,22826,23.634501,-102.552784
4,Referral,Chrome,desktop,False,Macintosh,1.662779e+17,San Francisco,Americas,United States,unknown.unknown,California,Northern America,-1.0,2,1.0,2.0,2.0,12.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),False,UNK,(none),/offer/2145,(direct),1508196701,1,1508196701,-0.068615,0.897805,-0.866025,645204,37.090240,-95.712891


###### Converting Catgorical data 

In [0]:
# Initialize label binarizer
lb = preprocessing.LabelBinarizer()

# Training data
train_features['trafficSource.isTrueDirect'] = lb.fit_transform(train_features['trafficSource.isTrueDirect'])
train_features['device.isMobile'] = lb.transform(train_features['device.isMobile'])
test_features['trafficSource.isTrueDirect'] = lb.fit_transform(test_features['trafficSource.isTrueDirect'])
test_features['device.isMobile'] = lb.transform(test_features['device.isMobile'])


In [0]:
train_features.head(5)

,channelGrouping,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime,year,month,day,country_dem,latitude,longitude
0,Referral,Chrome,desktop,0,Chrome OS,8.934117e+18,Cupertino,Americas,United States,unknown.unknown,California,Northern America,-1.0,2,0.0,2.0,2.0,28.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),0,UNK,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com,1508176307,6,1508176307,-0.068615,0.897805,-0.866025,645204,37.090240,-95.712891
1,Direct,Chrome,mobile,1,Android,7.992466e+18,not available in demo dataset,Americas,United States,windjammercable.net,not available in demo dataset,Northern America,-1.0,2,1.0,2.0,1.0,38.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),1,UNK,(none),UNK,(direct),1508201613,1,1508201613,-0.068615,0.897805,-0.866025,645204,37.090240,-95.712891
2,Organic Search,Chrome,desktop,0,Windows,9.075656e+18,not available in demo dataset,Asia,Turkey,unknown.unknown,not available in demo dataset,Western Asia,-1.0,2,1.0,2.0,1.0,1.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),0,(not provided),organic,UNK,google,1508169851,1,1508169851,-0.068615,0.897805,-0.866025,26282,38.963745,35.243322
3,Organic Search,Chrome,desktop,0,Windows,6.960673e+18,not available in demo dataset,Americas,Mexico,prod-infinitum.com.mx,not available in demo dataset,Central America,-1.0,2,1.0,2.0,1.0,52.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),0,(not provided),organic,UNK,google,1508190552,1,1508190552,-0.068615,0.897805,-0.866025,22826,23.634501,-102.552784
4,Referral,Chrome,desktop,0,Macintosh,1.662779e+17,San Francisco,Americas,United States,unknown.unknown,California,Northern America,-1.0,2,1.0,2.0,2.0,12.0,0.0,0.0,0.0,UNK,UNK,UNK,0.0,UNK,(not set),0,UNK,(none),/offer/2145,(direct),1508196701,1,1508196701,-0.068615,0.897805,-0.866025,645204,37.090240,-95.712891


In [0]:
for column in train_features.columns:
    if (train_features[column].dtype != np.int64 and train_features[column].dtype != np.float64):
        print("Converting column: {}".format(column))

        le = preprocessing.LabelEncoder()
        train_features[column] = le.fit_transform(train_features[column])
        del le
 
 
for column in test_features.columns:
    if (test_features[column].dtype != np.int64 and test_features[column].dtype != np.float64):
        print("Converting column: {}".format(column))

        le = preprocessing.LabelEncoder()
        test_features[column] = le.fit_transform(test_features[column])
        del le       


Converting column: channelGrouping
Converting column: device.browser
Converting column: device.deviceCategory
Converting column: device.operatingSystem
Converting column: geoNetwork.city
Converting column: geoNetwork.continent
Converting column: geoNetwork.country
Converting column: geoNetwork.networkDomain
Converting column: geoNetwork.region
Converting column: geoNetwork.subContinent
Converting column: trafficSource.adContent
Converting column: trafficSource.adwordsClickInfo.adNetworkType
Converting column: trafficSource.adwordsClickInfo.gclId
Converting column: trafficSource.adwordsClickInfo.slot
Converting column: trafficSource.campaign
Converting column: trafficSource.keyword
Converting column: trafficSource.medium
Converting column: trafficSource.referralPath
Converting column: trafficSource.source
Converting column: channelGrouping
Converting column: device.browser
Converting column: device.deviceCategory
Converting column: device.operatingSystem
Converting column: geoNetwork.ci

In [0]:
train_features.head(5)

,channelGrouping,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,fullVisitorId,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitId,visitNumber,visitStartTime,year,month,day,country_dem,latitude,longitude
0,6,35,0,0,3,8.934117e+18,220,2,218,36704,73,12,-1.0,2,0.0,2.0,2.0,28.0,0.0,0.0,0.0,58,3,53859,0.0,3,4,0,521,6,289,295,1508176307,6,1508176307,-0.068615,0.897805,-0.866025,645204,37.090240,-95.712891
1,2,35,1,1,1,7.992466e+18,955,2,218,38798,482,12,-1.0,2,1.0,2.0,1.0,38.0,0.0,0.0,0.0,58,3,53859,0.0,3,4,1,521,0,3089,0,1508201613,1,1508201613,-0.068615,0.897805,-0.866025,645204,37.090240,-95.712891
2,4,35,0,0,19,9.075656e+18,955,3,210,36704,482,21,-1.0,2,1.0,2.0,1.0,1.0,0.0,0.0,0.0,58,3,53859,0.0,3,4,0,7,5,3089,99,1508169851,1,1508169851,-0.068615,0.897805,-0.866025,26282,38.963745,35.243322
3,4,35,0,0,19,6.960673e+18,955,2,132,27691,482,3,-1.0,2,1.0,2.0,1.0,52.0,0.0,0.0,0.0,58,3,53859,0.0,3,4,0,7,5,3089,99,1508190552,1,1508190552,-0.068615,0.897805,-0.866025,22826,23.634501,-102.552784
4,6,35,0,0,7,1.662779e+17,749,2,218,36704,73,12,-1.0,2,1.0,2.0,2.0,12.0,0.0,0.0,0.0,58,3,53859,0.0,3,4,0,521,0,2179,0,1508196701,1,1508196701,-0.068615,0.897805,-0.866025,645204,37.090240,-95.712891


In [0]:
# Select subset of data to keep based on heatmap
keep_cols = ['totals.bounces',
             'totals.hits',
             'totals.newVisits',
             'totals.pageviews',
             'totals.sessionQualityDim',
             'totals.timeOnSite',
             'trafficSource.isTrueDirect',
             'visitNumber',
             'country_dem',
             'latitude',
             'longitude']
# Select subset of columns
train_feat = train_features[keep_cols]
test_feat = test_features[keep_cols]

In [0]:
# show the distribution
print('y_train class distribution')
unique_elements, counts_elements = np.unique(train_labels, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))
print('y_test class distribution')
unique_elements, counts_elements = np.unique(test_labels, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

print('Training Features Shape:', train_feat.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_feat.shape)
print('Testing Labels Shape:', test_labels.shape)


y_train class distribution
[[      0       1]
 [1024244  513259]]
y_test class distribution
[[     0      1]
 [219121 142308]]
Training Features Shape: (1537503, 11)
Training Labels Shape: (1537503,)
Testing Features Shape: (361429, 11)
Testing Labels Shape: (361429,)


## Random Forest 

In [0]:
#Training 
from sklearn.ensemble import RandomForestClassifier 

rf = RandomForestClassifier(n_estimators = 300,random_state = 42)

rf.fit(train_feat , train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
#Testing 
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix
from sklearn import metrics 

def adjusted_classes(y_score,t):
    return[1 if y>=t else 0 for y in y_score]

predictions = rf.predict_proba(test_feat)
y_score = predictions[:,1]

thresh = [0.5]
for t in thresh:
    y_pred_adj = adjusted_classes(y_score, t)

    print(confusion_matrix(test_labels,  y_pred_adj))
    print(accuracy_score(test_labels,  y_pred_adj))
    print(classification_report(test_labels,  y_pred_adj))
    


[[212361   6760]
 [ 42939  99369]]
0.8624930484272153
              precision    recall  f1-score   support

           0       0.83      0.97      0.90    219121
           1       0.94      0.70      0.80    142308

    accuracy                           0.86    361429
   macro avg       0.88      0.83      0.85    361429
weighted avg       0.87      0.86      0.86    361429



In [0]:
import pickle 
filename = "Random_Forest_CR1.sav"
pickle.dump(rf , open(filename,'wb'))

#XGB Model

In [0]:
#Training 
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators =100)
model.fit(train_feat , train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
#Testing 
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix
y_predProb = model.predict_proba(test_feat)
y_predv11 = model.predict(test_feat)
#Evaluating Predictions 
print(confusion_matrix(test_labels, y_predv11))
print(accuracy_score(test_labels, y_predv11))
print(classification_report(test_labels, y_predv11))

[[213625   5496]
 [ 43705  98603]]
0.8638709124060328
              precision    recall  f1-score   support

           0       0.83      0.97      0.90    219121
           1       0.95      0.69      0.80    142308

    accuracy                           0.86    361429
   macro avg       0.89      0.83      0.85    361429
weighted avg       0.88      0.86      0.86    361429



In [0]:
filename = "XGB_CR1.sav"
pickle.dump(model , open(filename,'wb'))

In [0]:
!ls


gdrive	Random_Forest_CR1.sav  sample_data  XGB_CR1.sav


In [0]:
!mv XGB_CR1.sav /content/gdrive/'My Drive'/'Colab Notebooks'


In [0]:
!mv Random_Forest_CR1.sav /content/gdrive/'My Drive'/'Colab Notebooks'


In [0]:
#Training 
from sklearn.ensemble import RandomForestClassifier 

rf = RandomForestClassifier(n_estimators = 100,random_state = 42)

rf.fit(train_feat , train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
#Testing 
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix
from sklearn import metrics 

def adjusted_classes(y_score,t):
    return[1 if y>=t else 0 for y in y_score]

predictions = rf.predict_proba(test_feat)
y_score = predictions[:,1]

thresh = [0.5]
for t in thresh:
    y_pred_adj = adjusted_classes(y_score, t)

    print(confusion_matrix(test_labels,  y_pred_adj))
    print(accuracy_score(test_labels,  y_pred_adj))
    print(classification_report(test_labels,  y_pred_adj))
    

[[211804   7317]
 [ 42593  99715]]
0.8619092546530577
              precision    recall  f1-score   support

           0       0.83      0.97      0.89    219121
           1       0.93      0.70      0.80    142308

    accuracy                           0.86    361429
   macro avg       0.88      0.83      0.85    361429
weighted avg       0.87      0.86      0.86    361429



In [0]:
#Loading the pickle model 
import pickle
xg1 = pickle.load(open("/content/gdrive/My Drive/Colab Notebooks/XGB_CR1.sav",'rb'))

In [0]:
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix
y_predProb = xg1.predict_proba(train_feat)
y_predv11 = xg1.predict(train_feat)
#Evaluating Predictions 
print(confusion_matrix(train_labels, y_predv11))
print(accuracy_score(train_labels, y_predv11))
print(classification_report(train_labels, y_predv11))

[[1013909   10335]
 [ 161250  352009]]
0.8884002177556727
              precision    recall  f1-score   support

           0       0.86      0.99      0.92   1024244
           1       0.97      0.69      0.80    513259

    accuracy                           0.89   1537503
   macro avg       0.92      0.84      0.86   1537503
weighted avg       0.90      0.89      0.88   1537503



In [0]:
rf1 = pickle.load(open("/content/gdrive/My Drive/Colab Notebooks/Random_Forest_CR1.sav",'rb'))

In [0]:
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix
y_predProb = rf1.predict_proba(train_feat)
y_predv11 = rf1.predict(train_feat)
#Evaluating Predictions 
print(confusion_matrix(train_labels, y_predv11))
print(accuracy_score(train_labels, y_predv11))
print(classification_report(train_labels, y_predv11))

[[1015657    8587]
 [ 105105  408154]]
0.9260541280244656
              precision    recall  f1-score   support

           0       0.91      0.99      0.95   1024244
           1       0.98      0.80      0.88    513259

    accuracy                           0.93   1537503
   macro avg       0.94      0.89      0.91   1537503
weighted avg       0.93      0.93      0.92   1537503

